In [1]:
import urllib.request
import datetime
import json
import ssl

context = ssl._create_unverified_context()

client_id = "_gJra_F38hGKFUClmgh3"
client_secret = "8wTUNbdKRV"






In [2]:
# [CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req,context=context)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode("utf-8")
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [3]:
# [CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    # query ~ display에 띄어쓰기 전부 제거시 URL에러는 해결, 다른 에러 발생
    # 48Line pDate의 출력방식 수정으로 다른 에러 해결
    parameters = "?query=%s&start=%s&display=%s" % (
        urllib.parse.quote(srcText),
        start,
        display,
    )

    url = base + node + parameters
    responseDecode = getRequestUrl(url)  # [CODE 1]

    if responseDecode == None:
        return None
    else:
        return json.loads(responseDecode)


In [4]:
# [CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post["title"]
    description = post["description"]
    org_link = post["originallink"]
    link = post["link"]

    pDate = datetime.datetime.strptime(post["pubDate"], "%a, %d %b %Y %H:%M:%S +0900")
    pDate = pDate.strftime("%Y-%m-%d %H:%M:%S")

    jsonResult.append(
        {
            "cnt": cnt,
            "title": title,
            "description": description,
            "org_link": org_link,
            "link": org_link,
            "pDate": pDate,
        }
    )
    return


In [5]:
# [CODE 0]
def main():
    node = "news"  # 크롤링할 대상
    srcText = input("검색어를 입력하세요: ")
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # [CODE 2]
    total = jsonResponse["total"]
    while (jsonResponse != None) and (jsonResponse["display"] != 0):
        for post in jsonResponse["items"]:
            cnt += 1
            getPostData(post, jsonResult, cnt)  # [CODE 3]

        start = jsonResponse["start"] + jsonResponse["display"]
        jsonResponse = getNaverSearch(node, srcText, start, 100)  # [CODE 2]

    print("전체 검색 : %d 건" % total)

    with open("%s_naver_%s.json" % (srcText, node), "w", encoding="utf8") as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print("%s_naver_%s.json SAVED" % (srcText, node))


In [6]:
if __name__ == "__main__":
    main()

[2022-10-17 21:49:17.615014] Url Request Success
[2022-10-17 21:49:17.819514] Url Request Success
[2022-10-17 21:49:18.020587] Url Request Success
[2022-10-17 21:49:18.225574] Url Request Success
[2022-10-17 21:49:18.437891] Url Request Success
[2022-10-17 21:49:18.659460] Url Request Success
[2022-10-17 21:49:18.869664] Url Request Success
[2022-10-17 21:49:19.114862] Url Request Success
[2022-10-17 21:49:19.356592] Url Request Success
[2022-10-17 21:49:19.579002] Url Request Success
HTTP Error 400: Bad Request
[2022-10-17 21:49:19.646524] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EB%82%A0%EC%94%A8&start=1001&display=100
전체 검색 : 2804583 건
가져온 데이터 : 1000 건
날씨_naver_news.json SAVED
